# Preprocess Review Dataset

Este dataset es uno de los más importantes, ya que una mala review puede ser un indicativo de que alguien va a dejar el servicio.

In [10]:
from pathlib import Path

import pandas as pd
import numpy as np

In [11]:
# Directorios donde se encuentran mis datos
BASE_DIR = Path.cwd().parent
DATA_DIR = (BASE_DIR / "data").resolve()

In [12]:
reviews = pd.read_csv(DATA_DIR/"raw"/"olist_order_reviews_dataset.csv")
df_reviews = pd.DataFrame(reviews)

Lo primero que quiero despues de importar mis datos es comprobar las columnas que tengo y si tengo datos repetidos, para poder preparar este dataset para el merge con mis otros datasets

In [13]:
print("\nreviews columns:\n", df_reviews.columns)


reviews columns:
 Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')


In [14]:
print(df_reviews['order_id'].value_counts().max())

3


 Lo primero que nos interesa es saber que tipos de datos tenemos por columna, ya que por el nombre parece que tendremos texto en las columnas y fechas.

In [15]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [16]:
df_reviews.sample(3)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
42324,b085317db08b64d8f36cbcc729b1aa20,8800d14eb6eb6031624538ca8750b921,1,NaN,Ainda não recebi o produto,2017-07-28 00:00:00,2017-07-28 19:07:43
43089,f210d277317161b0f34c2d52d4e504cb,82ee2df3a8f23cfa9208bde62204119e,5,NaN,que continue com essa perfeição,2018-04-19 00:00:00,2018-04-20 02:00:07
34346,c4aabbc3ca42032be47e779f24aa7bda,c6a5f1a0e18d1cad856f95794462ecbc,5,NaN,Olá. Recebi duas carretilhas apenas. Meu pedid...,2017-09-27 00:00:00,2017-09-28 15:18:02


Con esto podemos ver que tenemos una columna con el score de la review, que nos va a dar mucha información, sumado a el comentario de esta misma review.

Las fechas es algo que podremos descartar al final, ya que no nos proporciona información, junto con el review id.

Algo a tener en cuenta en el EDA es la cantidad de valores nulos en el titulo de las reviews y en las propias reviews.

In [17]:
# Comprobamos si hay orders duplicados
df_reviews["order_id"].value_counts().max()

np.int64(3)

Hay orders duplicados, esto puede significar que una persona editó su review para cambiar su nota.

Esto nos puede proporcionar información valiosa, podemos saber si la nota cambió, cuántas reviews totales puso un cliente por order y sacar una media de las notas.

En cuanto al texto de la review, nos quedaremos con el más reciente, en caso de querer utilizarlo más adelante.

Para comenzar, ordenaremos el dataset por orden cronológico, despues crearemos las columnas que nos pueden proporcionar más datos y despues haremos el merge

In [19]:
df_reviews["review_creation_date"] = pd.to_datetime(df_reviews["review_creation_date"])
df_reviews.sort_values("review_creation_date", inplace=True)

# Dejo esto aquí porque es mi primera vez usando .agg() así no me pierdo.

# df_total_rewiews = ( 
#     df_reviews
#         .groupby("order_id", as_index=False)["order_id"]
#         .count()
#         .rename(columns={"order_id": "total_reviews"})
# )

# df_mean_reviews = (
#     df_reviews
#         .groupby("order_id", as_index=False)["review_score"].mean()
#         .rename(columns={"review_score": "mean_score"})
# )

# df_worst_score = (
#     df_reviews
#     .groupby("order_id", as_index=False)["review_score"].min()
#     .rename(columns={"review_score": "min_score"})
# )

# df_last_score = (
#     df_reviews
#       .drop_duplicates(subset="order_id", keep="last")   # la última tras el sort
#       .loc[:, ["order_id", "review_score"]]
#       .rename(columns={"review_score": "last_score"})
# )

# Optimización de codigo usando función .agg()
agg_reviews = (df_reviews
                .groupby("order_id")
                .agg(
                    total_reviews = ("order_id", "size"),
                    worst_review = ("review_score", "min"),
                    mean_review_score = ("review_score", "mean"),
                    last_review = ("review_score", "last")
                )

)

agg_reviews.head(3)

,total_reviews,worst_review,mean_review_score,last_review
order_id,,,,
00010242fe8c5a6d1ba2dd792cb16214,1,5,5.0,5
00018f77f2f0320c557190d7a144bdd3,1,4,4.0,4
000229ec398224ef6ca0657da4fc703e,1,5,5.0,5


Una vez hecho esto, nuestro siguiente paso será unir este nuevo dataset al original y despues hacer drop de las columnas que no queremos mantener

In [ ]:
df_reviews.drop(columns=["review_id", "review_creation_date", "review_answer_timestamp"], inplace=True)
df_reviews.head()